In [1]:
!pip install transformers
!pip install sentencepiece
!pip install pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached pytorch-1.0.2.tar.gz (689 bytes)
  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
    Running setup.py install for pytorch ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-y2b_it2s/pytorch_4b5c888b12c64b80892f6dafb5c5ada3/setup.py'"'"'; __file__='"'"'/tmp/pip-install-y2b_it2s/pytorch_4b5c888b12c64b80892f6dafb5c5ada3/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'

In [2]:
import torch
import pandas as pd
from IPython.display import HTML, display
from transformers import T5Tokenizer, T5ForConditionalGeneration, Adafactor

In [3]:
def progress(loss, value, max = 100):
  return HTML(f""" Batch loss: {loss} <progress value = '{value}' max = '{max}', style = 'width: 100%'> {value}</progress>""")

In [4]:
text = ['1 search bar', '3 buttons', '2 checkbuttons', '1 search results']
text2 = ['1 search bar','airports','top']
text3 = '1 search bar | airports | top'
train_df = pd.read_csv('/content/drive/MyDrive/webNLG2020_train.csv', index_col=[0])

train_df

,prefix,input_text,target_text
0,webNLG,1_Decembrie_1918_University | nickname | Uab &...,The 1 Decembrie 1918 University is located in ...
1,webNLG,1_Decembrie_1918_University | nickname | Uab &...,The rector of the 1 Decembrie 1918 University ...
2,webNLG,1_Decembrie_1918_University | nickname | Uab &...,The 1 Decembrie 1918 University is located in ...
3,webNLG,AWH_Engineering_College | country | India && A...,"The AWH Engineering College in Kuttikkattoor, ..."
4,webNLG,AWH_Engineering_College | country | India && A...,The River Ganges runs through India in which c...
...,...,...,...
35208,webNLG,William_Anders | mission | Apollo_8,William Anders was a member of the Apollo 8 crew.
35209,webNLG,William_Anders | mission | Apollo_8,William Anders was a member of Apollo 8's crew.
35210,webNLG,William_Anders | selectedByNasa | 1963,William Anders was selected by NASA in 1963.
35211,webNLG,William_Anders | selectedByNasa | 1963,William Anders joined NASA in 1963.


In [9]:
if torch.cuda.is_available():
  dev = torch.device('cuda:0')
  print('running on GPU...')
else:
  dev = torch.device('cpu')
  print('running on CPU')

running on GPU...


In [10]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base', return_dict = True)
model.train()
model.to(dev)

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [11]:
optimizer = Adafactor(model.parameters(), lr = 1e-3, eps = (1e-30, 1e-3), clip_threshold = 1, decay_rate = -0.8, beta1 = None, weight_decay = 0, relative_step = False, scale_parameter = False, warmup_init = False)

In [12]:
batch_size = 8
num_of_batches = int(len(train_df) / batch_size)
epochs = 4
loss_per_10_steps = []
for epoch in range(1, epochs + 1):
  print('epoch: ',epoch)
  running_loss = 0
  out = display(progress(1, num_of_batches + 1), display_id = True)
  for i in range(num_of_batches):
    input_batch = []
    label_batch = []
    new_df = train_df[i*batch_size:(i + 1)*batch_size]
    for index, row in new_df.iterrows():
      input = 'WebNLG: '+row['input_text']+'</s>'
      label = row['target_text']+'</s>'
      input_batch.append(input)
      label_batch.append(label)
    input_batch = tokenizer.batch_encode_plus(input_batch, padding = True, max_length = 400, return_tensors = 'pt')['input_ids']
    label_batch = tokenizer.batch_encode_plus(label_batch, padding = True, max_length = 400, return_tensors = 'pt')['input_ids']
    input_batch = input_batch.to(dev)
    label_batch = label_batch.to(dev)
    
    optimizer.zero_grad()

    outputs = model(input_ids = input_batch, labels = label_batch)
    loss = outputs.loss
    loss_num = loss.item()
    logits = outputs.logits
    running_loss += loss_num
    if i%10 == 0:
      loss_per_10_steps.append(loss_num)
    out.update(progress(loss_num, i, num_of_batches+1))

    loss.backward()

    optimizer.step()

    running_loss /= int(num_of_batches)
    print(f'epoch: {epoch} , running loss: {running_loss}')

epoch:  1


/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2354: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


epoch: 1 , running loss: 0.001152100765659061
epoch: 1 , running loss: 0.0007675188266995228
epoch: 1 , running loss: 0.0003163084665980239
epoch: 1 , running loss: 0.0003359815199393402
epoch: 1 , running loss: 0.0003569069901497036
epoch: 1 , running loss: 0.0002917507987062712
epoch: 1 , running loss: 0.0003089730449515952
epoch: 1 , running loss: 0.0003334488565816325
epoch: 1 , running loss: 0.0002036927091386234
epoch: 1 , running loss: 0.00024439341853162323
epoch: 1 , running loss: 0.0001652545242190887
epoch: 1 , running loss: 0.00016698995617686607
epoch: 1 , running loss: 0.0001586588644510697
epoch: 1 , running loss: 0.00025195905678001933
epoch: 1 , running loss: 0.000289315056669725
epoch: 1 , running loss: 0.0002605471644231147
epoch: 1 , running loss: 0.0003993112922291152
epoch: 1 , running loss: 0.0004713240513257802
epoch: 1 , running loss: 0.0002443985123361179
epoch: 1 , running loss: 0.0001544960969177484
epoch: 1 , running loss: 0.0002486312480709216
epoch: 1 , r

epoch: 2 , running loss: 0.000271799563819835
epoch: 2 , running loss: 0.00017516444038720316
epoch: 2 , running loss: 0.00013900446165385254
epoch: 2 , running loss: 9.870135884205515e-05
epoch: 2 , running loss: 0.00013851872865972218
epoch: 2 , running loss: 0.00015937877008290825
epoch: 2 , running loss: 0.0001364144584972176
epoch: 2 , running loss: 0.00016290089801337457
epoch: 2 , running loss: 0.00011075434188999558
epoch: 2 , running loss: 9.123699159516048e-05
epoch: 2 , running loss: 0.00012139213465359263
epoch: 2 , running loss: 0.00010963535777527028
epoch: 2 , running loss: 0.00010298971419168955
epoch: 2 , running loss: 0.00014436931455009082
epoch: 2 , running loss: 0.0001582034325330695
epoch: 2 , running loss: 0.0001857968224684878
epoch: 2 , running loss: 0.0001610560997026866
epoch: 2 , running loss: 0.00016235640353129402
epoch: 2 , running loss: 0.00014158577368128636
epoch: 2 , running loss: 0.00013412928173726183
epoch: 2 , running loss: 0.00011381773476181927


epoch: 3 , running loss: 0.0002384256758599952
epoch: 3 , running loss: 0.00013886188110560685
epoch: 3 , running loss: 0.00012184624610954665
epoch: 3 , running loss: 9.036197199696019e-05
epoch: 3 , running loss: 0.00012246900435717233
epoch: 3 , running loss: 0.00013568973260828654
epoch: 3 , running loss: 9.994792867742855e-05
epoch: 3 , running loss: 0.0001130229117949448
epoch: 3 , running loss: 9.196330636129102e-05
epoch: 3 , running loss: 6.88751225504935e-05
epoch: 3 , running loss: 9.553837728522112e-05
epoch: 3 , running loss: 0.00011596319795158668
epoch: 3 , running loss: 0.00010048968686891838
epoch: 3 , running loss: 0.00012881987065601464
epoch: 3 , running loss: 0.00013054957417453492
epoch: 3 , running loss: 0.00015966013894524032
epoch: 3 , running loss: 0.00012880062158525537
epoch: 3 , running loss: 0.00015888123798101213
epoch: 3 , running loss: 0.00012971958316702371
epoch: 3 , running loss: 0.00010771623599134101
epoch: 3 , running loss: 0.0001039145247791325
e

epoch: 3 , running loss: 0.00011436305740274413
epoch: 3 , running loss: 0.00018316678926252482
epoch: 3 , running loss: 8.871885962535407e-05
epoch: 3 , running loss: 7.49417007290595e-05
epoch: 3 , running loss: 9.020050817852018e-05
epoch: 3 , running loss: 0.00013746544753832077
epoch: 3 , running loss: 9.116826054597936e-05
epoch: 3 , running loss: 0.00014010837025545323
epoch: 3 , running loss: 0.00011733085111269358
epoch: 3 , running loss: 0.0001296930096013547
epoch: 3 , running loss: 0.0001576064485068508
epoch: 3 , running loss: 0.00020063121432892914
epoch: 3 , running loss: 0.00018576384177425642
epoch: 3 , running loss: 7.495600455213092e-05
epoch: 3 , running loss: 6.13837919985924e-05
epoch: 3 , running loss: 0.00013943133814036995
epoch: 3 , running loss: 0.0001469442377640473
epoch: 3 , running loss: 0.00015937083846108427
epoch: 3 , running loss: 0.0001544044549098012
epoch: 3 , running loss: 0.00012551858512483847
epoch: 3 , running loss: 0.00012777267779275
epoch: 

epoch: 4 , running loss: 0.00019972768866780184
epoch: 4 , running loss: 0.00012015740936131783
epoch: 4 , running loss: 0.00010598174994777445
epoch: 4 , running loss: 7.115441026742515e-05
epoch: 4 , running loss: 0.00010328162115871094
epoch: 4 , running loss: 0.0001372197353423686
epoch: 4 , running loss: 9.049817726925895e-05
epoch: 4 , running loss: 0.00011986380715917946
epoch: 4 , running loss: 8.897468086730147e-05
epoch: 4 , running loss: 9.098061575127149e-05
epoch: 4 , running loss: 8.502769027943601e-05
epoch: 4 , running loss: 8.680859914296487e-05
epoch: 4 , running loss: 9.001942698586719e-05
epoch: 4 , running loss: 0.00012012649513394389
epoch: 4 , running loss: 0.00012327542589074541
epoch: 4 , running loss: 0.00014769893128187245
epoch: 4 , running loss: 0.0001254735193541998
epoch: 4 , running loss: 0.00013521757345172074
epoch: 4 , running loss: 0.00010719703146930818
epoch: 4 , running loss: 0.0001010292900899507
epoch: 4 , running loss: 9.432210878661379e-05
epo

In [24]:
# !wget -P '/content/drive/MyDrive/t5-model' https://s3.amazonaws.com/models.huggingface.co/bert/t5-base-config.json 
torch.save(model.state_dict(),r'/content/drive/MyDrive/t5-model/pytorch_model.bin')


In [37]:
# tokenizer = T5Tokenizer.from_pretrained('t5-base')
# model = T5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/t5-model/t5-base-config.json', return_dict = True)
def kt(text, model = model, tokenizer = tokenizer):
  model.eval()
  input_ids = tokenizer.encode(f'WebNLG: {text} <\s>', return_tensors = 'pt')
  outputs = model.generate(input_ids.cuda())
  return tokenizer.decode(outputs[0])

In [39]:
text = 'search bar | quantity | 1 && search bar | subject | airports && search bar | location | top'
kt(text)
# model.eval()
# input_ids = tokenizer.encode(f'WebNLG: {text} <\s>')
# outputs = model.generate(input_ids)
# tokenizer.decode(outputs[0])

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


"<pad>The number of the people that are found in the city of '1's''"

In [42]:
kt('Lakshmi | study | Computer Science && Lakshmi | born | April 5th 1996 && Lakshmi | lives | Norfolk')

'<pad> Lakshmi was born April 5th 1996 and lives in Norfolk.</s>'